In [ ]:
import os
os.environ['CASTLE_BACKEND'] ='pytorch'
import dill
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import shap
import matplotlib.pyplot as plt

import random
from castle.algorithms.ges.ges import GES

In [ ]:
def find_valid_edges(G):
    edge_list = []
    
    for cause in G.index:
        for target in G.columns:
            if G.loc[cause, target] == 1:  
                edge_list.append({'Cause': cause, 'Target': target})
    
    edges = pd.DataFrame(edge_list, columns=['Cause', 'Target'])
    
    return edges

def sclar_coder(rundf,num_v,cate_v):
    scaler = MinMaxScaler()
    rundf[num_v] = scaler.fit_transform(rundf[num_v])
    
    label_encoder = LabelEncoder()
    for col in cate_v:
        rundf[col] = label_encoder.fit_transform(rundf[col])

    return rundf

In [ ]:
dag_path = '/DAGs/'

III_IV = pd.read_csv('/New_IV_III_downsampled.csv')

basic = ['GENDER', 'ADMISSION_TYPE', 'FIRST_CAREUNIT', 'AGE']
groups = ['Group_Va_uti','Group_AKI', 'Group_CKD', 'Group_PCOS', 'Group_Neoplasm_ovary', 'Group_Endometriosis', 'Group_Leiomyoma']

full_Diag = groups + ['Diag_Acinetobacter spp.', 'Diag_Enterobacteriaceae','Diag_Enterococcus spp.', 'Diag_Pseudomonas aeruginosa', 'Diag_Staphylococcus aureus','Diag_MDRObeforeICU','Diag_ATE_filtered','Diag_Acute respiratory failure','Diag_Do not resuscitate status', 'Diag_Encounter for palliative care','Diag_Hyperosmolality and|or hypernatremia','Diag_MYOCARDIAL INFARCT', 'Diag_CONGESTIVE HEART FAILURE', 'Diag_PERIPHERAL VASCULAR DISEASE', 'Diag_CEREBROVASCULAR DISEASE', 'Diag_DEMENTIA', 'Diag_CHRONIC PULMONARY DISEASE', 'Diag_RHEUMATIC DISEASE', 'Diag_PEPTIC ULCER DISEASE', 'Diag_MILD LIVER DISEASE', 'Diag_DIABETES WITHOUT CC', 'Diag_DIABETES WITH CC', 'Diag_PARAPLEGIA', 'Diag_RENAL DISEASE', 'Diag_MALIGNANT CANCER', 'Diag_SEVERE LIVER DISEASE', 'Diag_METASTATIC SOLID TUMOR', 'Diag_AIDS', 'Diag_CHARLSON COMORBIDITY INDEX','Diag_9_2113', 'Diag_9_39891', 'Diag_9_43310', 'Diag_9_5570', 'Diag_9_64891', 'Diag_9_66411', 'Diag_9_6930', 'Diag_9_7464', 'Diag_9_7470', 'Diag_9_86121', 'Diag_9_99702', 'Diag_9_9971', 'Diag_9_9973', 'Diag_9_99731', 'Diag_9_9974', 'Diag_9_9975', 'Diag_10_A04', 'Diag_10_A40', 'Diag_10_B15', 'Diag_10_B20', 'Diag_10_B37', 'Diag_10_B95', 'Diag_10_C22', 'Diag_10_C33', 'Diag_10_C61', 'Diag_10_C78', 'Diag_10_C79', 'Diag_10_C82', 'Diag_10_D50', 'Diag_10_D61', 'Diag_10_D64', 'Diag_10_D66', 'Diag_10_D69', 'Diag_10_D70', 'Diag_10_E11', 'Diag_10_E22', 'Diag_10_E28', 'Diag_10_E43', 'Diag_10_E44', 'Diag_10_E66', 'Diag_10_E78', 'Diag_10_E83', 'Diag_9_E8490', 'Diag_9_E8497', 'Diag_9_E8498', 'Diag_9_E8499', 'Diag_10_E87', 'Diag_9_E8780', 'Diag_9_E8781', 'Diag_9_E8782', 'Diag_9_E8786', 'Diag_9_E8788', 'Diag_9_E8790', 'Diag_9_E8798', 'Diag_9_E8859', 'Diag_10_E89', 'Diag_9_E9320', 'Diag_9_E9331', 'Diag_9_E9342', 'Diag_9_E9478', 'Diag_10_F04', 'Diag_10_F05', 'Diag_10_F10', 'Diag_10_F19', 'Diag_10_F30', 'Diag_10_F32', 'Diag_10_F41', 'Diag_10_F43', 'Diag_10_G20', 'Diag_10_G30', 'Diag_10_G35', 'Diag_10_G40', 'Diag_10_G43', 'Diag_10_G44', 'Diag_10_G47', 'Diag_10_G60', 'Diag_10_G61', 'Diag_10_G81', 'Diag_10_G89', 'Diag_10_G93', 'Diag_10_G97', 'Diag_10_H35', 'Diag_10_H40', 'Diag_10_I07', 'Diag_10_I08', 'Diag_10_I10', 'Diag_10_I12', 'Diag_10_I20', 'Diag_10_I21', 'Diag_10_I24', 'Diag_10_I25', 'Diag_10_I26', 'Diag_10_I27', 'Diag_10_I31', 'Diag_10_I33', 'Diag_10_I34', 'Diag_10_I42', 'Diag_10_I44', 'Diag_10_I47', 'Diag_10_I50', 'Diag_10_I60', 'Diag_10_I61', 'Diag_10_I62', 'Diag_10_I66', 'Diag_10_I67', 'Diag_10_I69', 'Diag_10_I70', 'Diag_10_I71', 'Diag_10_I73', 'Diag_10_I82', 'Diag_10_I85', 'Diag_10_I95', 'Diag_10_J15', 'Diag_10_J41', 'Diag_10_J43', 'Diag_10_J44', 'Diag_10_J45', 'Diag_10_J69', 'Diag_10_J84', 'Diag_10_J93', 'Diag_10_J95', 'Diag_10_J98', 'Diag_10_K22', 'Diag_10_K29', 'Diag_10_K31', 'Diag_10_K41', 'Diag_10_K56', 'Diag_10_K57', 'Diag_10_K59', 'Diag_10_K66', 'Diag_10_K70', 'Diag_10_K75', 'Diag_10_K76', 'Diag_10_K80', 'Diag_10_K85', 'Diag_10_K91', 'Diag_10_K92', 'Diag_10_L03', 'Diag_10_L89', 'Diag_10_M06', 'Diag_10_M10', 'Diag_10_M15', 'Diag_10_M32', 'Diag_10_M48', 'Diag_10_M60', 'Diag_10_M79', 'Diag_10_M81', 'Diag_10_N05', 'Diag_10_N13', 'Diag_10_N23', 'Diag_10_N28', 'Diag_10_N39', 'Diag_10_N72', 'Diag_10_N92', 'Diag_10_O75', 'Diag_10_Q20', 'Diag_10_R00', 'Diag_10_R06', 'Diag_10_R09', 'Diag_10_R10', 'Diag_10_R19', 'Diag_10_R45', 'Diag_10_R58', 'Diag_10_R63', 'Diag_10_R68', 'Diag_10_R69', 'Diag_10_R78', 'Diag_10_S01', 'Diag_10_S06', 'Diag_10_S12', 'Diag_10_S27', 'Diag_10_T78', 'Diag_10_T81', 'Diag_10_T82', 'Diag_9_V1251', 'Diag_9_V1259', 'Diag_9_V420', 'Diag_9_V422', 'Diag_9_V433', 'Diag_9_V4365', 'Diag_9_V440', 'Diag_9_V441', 'Diag_9_V462', 'Diag_9_V6284', 'Diag_10_Z00', 'Diag_10_Z16', 'Diag_10_Z21', 'Diag_10_Z23', 'Diag_10_Z51', 'Diag_10_Z80', 'Diag_10_Z81', 'Diag_10_Z85', 'Diag_10_Z87', 'Diag_10_Z91', 'Diag_10_Z95', 'Diag_Personal history of antineoplastic chemotherapy', 'Diag_Personal history of pulmonary embolism', 'Diag_Personal history of transient ischemic attack (TIA), and cerebral infarction without residual deficits', 'Diag_Pneumonia, unspecified organism', 'Diag_10_A419', 'Diag_10_R6521', 'Diag_10_Z794', 'Diag_10_D62']
full_Proc = ['Pro_0BH17EZ', 'Pro_02H633Z', 'Pro_0DJ08ZZ', 'Pro_3E0G76Z', 'Pro_5A1955Z', 'Pro_0W9G3ZX', 'Pro_B211YZZ','Pro_0014', 'Pro_0017', 'Pro_0040', 'Pro_0045', 'Pro_0066', 'Pro_0159', 'Pro_0331', 'Pro_0741', 'Pro_311', 'Pro_3322', 'Pro_3323', 'Pro_3324', 'Pro_3404', 'Pro_3491', 'Pro_3521', 'Pro_3522', 'Pro_3606', 'Pro_3607', 'Pro_3611', 'Pro_3612', 'Pro_3613', 'Pro_3615', 'Pro_3721', 'Pro_3722', 'Pro_3723', 'Pro_3761', 'Pro_3845', 'Pro_387', 'Pro_3891', 'Pro_3893', 'Pro_3895', 'Pro_3897', 'Pro_3899', 'Pro_3950', 'Pro_3961', 'Pro_3995', 'Pro_4311', 'Pro_4443', 'Pro_4513', 'Pro_4516', 'Pro_4523', 'Pro_5185', 'Pro_5459', 'Pro_5491', 'Pro_7309', 'Pro_734', 'Pro_7359', 'Pro_7569', 'Pro_8051', 'Pro_8162', 'Pro_8622', 'Pro_8659', 'Pro_8744', 'Pro_8841', 'Pro_8842', 'Pro_8847', 'Pro_8848', 'Pro_8852', 'Pro_8853', 'Pro_8855', 'Pro_8856', 'Pro_8872', 'Pro_8938', 'Pro_8964', 'Pro_9390', 'Pro_9604', 'Pro_9605', 'Pro_9607', 'Pro_966', 'Pro_9671', 'Pro_9672', 'Pro_9904', 'Pro_9905', 'Pro_9907', 'Pro_9910', 'Pro_9915', 'Pro_9920', 'Pro_9925', 'Pro_9955', 'Pro_9960', 'Pro_9962']

Diag = groups + ['Diag_MDRObeforeICU', 'Diag_ATE_filtered', 'Diag_Acute respiratory failure', 'Diag_Do not resuscitate status', 'Diag_Encounter for palliative care', 'Diag_Hyperosmolality and|or hypernatremia', 'Diag_CONGESTIVE HEART FAILURE', 'Diag_DEMENTIA', 'Diag_RHEUMATIC DISEASE', 'Diag_RENAL DISEASE', 'Diag_MALIGNANT CANCER', 'Diag_METASTATIC SOLID TUMOR', 'Diag_AIDS', 'Diag_CHARLSON COMORBIDITY INDEX', 'Diag_MYOCARDIAL INFARCT', 'Diag_DIABETES WITHOUT CC', 'Diag_DIABETES WITH CC', 'Diag_CHRONIC PULMONARY DISEASE', 'Diag_SEVERE LIVER DISEASE', 'Diag_10_B37', 'Diag_10_D62', 'Diag_10_R00', 'Diag_10_D66', 'Diag_10_T81', 'Diag_MILD LIVER DISEASE', 'Diag_10_G20', 'Diag_10_I10', 'Diag_10_K70', 'Diag_9_V440', 'Diag_10_R6521', 'Diag_10_D50', 'Diag_10_K75', 'Diag_10_A419', 'Diag_10_E43', 'Diag_10_J69', 'Diag_9_99731', 'Diag_10_A40', 'Diag_10_C79', 'Diag_10_K76', 'Diag_10_I82', 'Diag_10_E11', 'Diag_10_E87', 'Diag_10_F05', 'Diag_10_I50', 'Diag_9_E8780', 'Diag_10_J44', 'Diag_10_N28', 'Diag_10_D70', 'Diag_10_I21', 'Diag_10_Z95', 'Diag_10_J93', 'Diag_10_I26', 'Diag_10_G35', 'Diag_9_E9342', 'Diag_9_V422', 'Diag_10_J15', 'Diag_10_E89', 'Diag_10_G97']
Proc = ['Pro_9672', 'Pro_9671', 'Pro_9960', 'Pro_5491', 'Pro_3961', 'Pro_3897', 'Pro_02H633Z', 'Pro_0BH17EZ', 'Pro_3895', 'Pro_0DJ08ZZ', 'Pro_3491', 'Pro_966', 'Pro_5A1955Z', 'Pro_3E0G76Z', 'Pro_3995', 'Pro_8964', 'Pro_3522', 'Pro_3521', 'Pro_0066', 'Pro_3611', 'Pro_3607', 'Pro_3612', 'Pro_3613', 'Pro_3615', 'Pro_9904', 'Pro_9905', 'Pro_9910']

Med = ['Med_Acetaminophen', 'Med_Amiodarone', 'Med_Ampicillin-Sulbactam', 'Med_Aspirin', 'Med_Atorvastatin', 'Med_Azithromycin', 'Med_Calcium Gluconate', 'Med_Cefepime', 'Med_Ceftazidime', 'Med_Ceftriaxone', 'Med_Ciprofloxacin', 'Med_Clindamycin', 'Med_Dexamethasone', 'Med_Dexmedetomidine', 'Med_Diltiazem', 'Med_Famotidine', 'Med_Fentanyl Citrate', 'Med_Folic Acid', 'Med_Furosemide', 'Med_Gabapentin', 'Med_Glucagon', 'Med_Haloperidol', 'Med_Heparin', 'Med_Hydromorphone', 'Med_Insulin', 'Med_Lactulose', 'Med_Levetiracetam', 'Med_Levofloxacin', 'Med_Linezolid', 'Med_Lorazepam', 'Med_Magnesium Sulfate', 'Med_Meropenem', 'Med_Metoclopramide', 'Med_Metoprolol', 'Med_Metronidazole', 'Med_Midazolam', 'Med_Morphine Sulfate', 'Med_Nitroglycerin', 'Med_Norepinephrine', 'Med_Omeprazole', 'Med_Ondansetron', 'Med_Oxycodone', 'Med_Pantoprazole', 'Med_Phenylephrine', 'Med_Piperacillin-Tazobactam', 'Med_Potassium Chloride', 'Med_Prednisone', 'Med_Propofol', 'Med_Ranitidine', 'Med_Simvastatin', 'Med_Sodium Bicarbonate', 'Med_Tacrolimus', 'Med_Thiamine', 'Med_Vancomycin', 'Med_Warfarin']
TS = ['TS_Heart Rate', 'TS_NSBP', 'TS_NDBP', 'TS_NMBP', 'TS_Respiratory_Rate', 'TS_Temperature_C', 'TS_O2 Flow', 'TS_SpO2 Desat Limit', 'TS_Admission_Weight', 'TS_SpO2', 'TS_Pain Level Response', 'TS_Pain', 'TS_Respiratory_Rate_Total', 'TS_Total_PEEP', 'TS_History of falling (within 3 mnths)', 'TS_Calculated Total CO2', 'TS_Base Excess', 'TS_pCO2', 'TS_pH', 'TS_Lactate', 'TS_PaO2', 'TS_Lactic Acid', 'TS_Self ADL', 'TS_ETOH', 'TS_ASBP', 'TS_ADBP', 'TS_MAP', 'TS_Arterial_PaCO2', 'TS_PH (Arterial)', 'TS_Arterial Base Excess', 'TS_Ionized_Calcium', 'TS_Arterial_TCO2', 'TS_Potassium (whole blood)', 'TS_Glucose_WholeBlood', 'TS_PEEP_Set', 'TS_Daily Weight', 'TS_Resp_Spontaneous', 'TS_Peak Insp. Pressure', 'TS_Mean Airway Pressure', 'TS_Visual / hearing deficit', 'TS_Mental status', 'TS_CAM-ICU MS Change', 'TS_Skin Care', 'TS_Back Care', 'TS_Plateau Pressure', 'TS_SaO2', 'TS_FiO2', 'TS_Hemoglobin', 'TS_Hematocrit (serum)', 'TS_WBC', 'TS_Chloride (serum)', 'TS_Creatinine', 'TS_Glucose (serum)', 'TS_Magnesium', 'TS_Sodium (serum)', 'TS_BUN', 'TS_Calcium non-ionized', 'TS_Phosphorous', 'TS_Anion_Gap', 'TS_Potassium (serum)', 'TS_HCO3 (serum)', 'TS_PLT', 'TS_PT', 'TS_PTT', 'TS_INR', 'TS_Hematocrit, Calculated', 'TS_Hematocrit', 'TS_HCO3', 'TS_Creatine Kinase, MB Isoenzyme', 'TS_Phosphate', 'TS_Calcium', 'TS_Chloride', 'TS_Glucose', 'TS_Potassium', 'TS_Sodium', 'TS_Alkaline Phosphatase', 'TS_AST', 'TS_LDH', 'TS_ALT', 'TS_Alkaline Phosphate', 'TS_Creatine_Kinase', 'TS_Bilirubin_total', 'TS_Troponin-T', 'TS_Albumin', 'TS_RDW', 'TS_RBC', 'TS_INR(PT)', 'TS_MCV', 'TS_Basophils', 'TS_Eosinophils', 'TS_Monocytes', 'TS_Neutrophils', 'TS_MCHC', 'TS_MCH', 'TS_Lymphocytes', 'TS_Strength L Arm', 'TS_Strength L Leg', 'TS_Strength R Leg', 'TS_Strength R Arm', 'TS_Differential-Lymphs', 'TS_Potassium_WholeBlood', 'TS_Sodium_WholeBlood', 'TS_Pain_Presence', 'TS_Tidal Volume (observed)', 'TS_Inspiratory Time', 'TS_Specific Gravity', 'TS_Difficulty swallowing', 'TS_Differential-Neuts', 'TS_Fingerstick_Glucose', 'TS_Unintentional weight loss >10 lbs.', 'TS_Oxygen', 'TS_Bed Bath', 'TS_CAM-ICU Inattention', 'TS_PH (Venous)', 'TS_GCS_Total']

label = ['LOS_Hospital','DIEINHOSPITAL','Readmission_30','Multiple_ICUs','sepsis_all', 'FirstICU24_AKI_ALL','ICU_within_12hr_of_admit']

print('Diag:',len(Diag), 'full_Diag:', len(full_Diag),'Proc:',len(Proc),'full_Proc:',len(full_Proc),'Med:',len(Med),'TS:',len(TS),'label:',len(label))

num_v = Med + TS + ['AGE']
cate_v = ['ADMISSION_TYPE','FIRST_CAREUNIT'] + Diag + Proc + label

III_IV[Diag + Proc + Med] = III_IV[Diag + Proc + Med].fillna(0)
III_IV[TS] = III_IV[TS].fillna(III_IV[TS].median())

rundf = sclar_coder(III_IV,num_v,cate_v)
rundf.shape

In [ ]:
feat = [label[4]] + Diag + TS + Med + Proc 
print(len(feat))

gmodel = GES(criterion='bic', method='scatter')
random.shuffle(feat)
print(len(np.array(Test[feat])))
gmodel.learn(np.array(Test[feat]))
df1 = find_valid_edges(pd.DataFrame(gmodel.causal_matrix.tolist(),index=feat,columns=feat))